In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

table_path = "workspace.breast_cancer.breast_cancer"

In [0]:
df_raw = spark.read.table(table_path)

In [0]:
display(df_raw)
df_raw.printSchema()

In [0]:
%python
df_clean = df_raw.drop("id") \
    .withColumn(
        "label",
        F.when(F.col("diagnosis") == "M", 1).otherwise(0)
    ) \
    .drop("diagnosis")

In [0]:
display(df_clean)
df_clean.printSchema()

In [0]:

expressoes_agregacao = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_clean.columns]

df_clean.select(expressoes_agregacao).show()

In [0]:
cols_to_compare = ["radius_mean", "area_mean", "concavity_mean", "label"]
df_clean.select(cols_to_compare).summary("min", "max", "mean", "stddev").show()

In [0]:
df_grouped = df_clean.groupBy("label").count()
display(df_grouped)

Databricks visualization. Run in Databricks to view.

"Conforme a teoria de SVM, identifiquei disparidade de escalas (ex: area_mean vs smoothness_mean). Isso torna o passo de Standard Scaling obrigatório para evitar o viés do modelo."

In [0]:
#separação das colunas numéricas para o pipeline
feature_cols = [c for c in df_clean.columns if c != 'label']
print(feature_cols)

In [0]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features_raw"
)
scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withStd=True,
    withMean=True
)
pipeline_preparacao = Pipeline(stages=[assembler, scaler])

modelo_preparacao = pipeline_preparacao.fit(df_clean)

df_final = modelo_preparacao.transform(df_clean)

display(
    df_final.select(
        "features_raw",
        "features",
        "label"
    ).limit(5)
)

In [0]:
# Divisão dos dados em 70% Treino e 30% Teste
# seed=42 garante que a divisão seja sempre a mesma (reprodutibilidade)
train_data, test_data = df_final.randomSplit([0.7, 0.3], seed=42)

print(f"Registros para Treinamento: {train_data.count()}")
print(f"Registros para Teste (Prova Real): {test_data.count()}")

In [0]:
#  TREINAMENTO DO MODELO SVM
from pyspark.ml.classification import LinearSVC

# 1. Instanciando o Algoritmo (A "folha em branco")
# maxIter=100: O SVM usa Otimização (Gradiente Descendente). Damos 100 chances para ele achar a linha ideal.
# regParam=0.1: Equivalente a um C moderado. Estamos pedindo uma margem nem muito rígida, nem muito solta.
svm = LinearSVC(
    featuresCol="features",
    labelCol="label",        
    maxIter=100, 
    regParam=0.1
)

# 2. O Treinamento (FIT)
# Aqui o Spark distribui os dados e calcula a melhor reta (hiperplano) que separa as classes.
print("Treinando o modelo SVM...")
svm_model = svm.fit(train_data)

# 3. Extraindo os "Segredos" do Modelo (Coeficientes)
# O SVM é uma "caixa branca". Podemos ver exatamente quais features ele achou mais importantes.
# Os coeficientes são os pesos (w) da equação da reta w*x + b = 0
print(f"Intercepto (b): {svm_model.intercept}")
print(f"Número de pesos (w) aprendidos: {len(svm_model.coefficients)}")

# 4. Fazendo Previsões (TRANSFORM)
# Agora aplicamos a "prova" (dados de teste) para ver como ele se sai.
predictions = svm_model.transform(test_data)

# 5. Visualizando o Resultado
# 'rawPrediction': A distância do ponto até o hiperplano (a "pontuação" de confiança).
# 'prediction': A decisão final (0 ou 1).
print("--- Resultados nos Dados de Teste ---")
display(predictions.select("label", "rawPrediction", "prediction"))

In [0]:
# --- FASE 5: AVALIAÇÃO DO MODELO ---
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# 1. Configurando o Avaliador (O "Professor" que dá a nota)
# Usamos MulticlassClassificationEvaluator mesmo para binário para pegar a Acurácia exata
evaluator_acc = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="accuracy"
)

evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="f1"
)

# 2. Calculando as Notas
accuracy = evaluator_acc.evaluate(predictions)
f1_score = evaluator_f1.evaluate(predictions)

print("="*30)
print(f"🎯 Acurácia Global: {accuracy:.2%}")
print(f"⚖️  F1-Score (Equilíbrio): {f1_score:.2%}")
print("="*30)

# 3. Matriz de Confusão (Onde o modelo errou?)
# Isso mostra: Falsos Positivos vs Falsos Negativos
print("\n--- Matriz de Confusão ---")
# Agrupamos pelo Gabarito (label) e pelo Chute (prediction)
confusion_matrix = predictions.groupBy("label", "prediction").count() \
                              .orderBy("label", "prediction")

display(confusion_matrix)

In [0]:
# --- FASE DE AVALIAÇÃO AVANÇADA (JEITO PRO, SUPORTADO NO SERVERLESS) ---
from pyspark.sql import functions as F

# 1. Matriz de Confusão via DataFrame
confusion_df = predictions.groupBy("label", "prediction").count().orderBy("label", "prediction")
display(confusion_df)

# 2. Calculando Recall e Precision para a classe 'Maligno' (1)
TP = predictions.filter((F.col("label") == 1) & (F.col("prediction") == 1)).count()
FN = predictions.filter((F.col("label") == 1) & (F.col("prediction") == 0)).count()
FP = predictions.filter((F.col("label") == 0) & (F.col("prediction") == 1)).count()

recall_maligno = TP / (TP + FN) if (TP + FN) > 0 else 0
precision_maligno = TP / (TP + FP) if (TP + FP) > 0 else 0

print("\n" + "="*40)
print("📊 DETALHAMENTO DA CLASSE 'MALIGNO' (1.0)")
print("="*40)
print(f"🚨 Recall (Sensibilidade): {recall_maligno:.2%} (De todos os cânceres, achamos X%)")
print(f"🎯 Precision (Precisão):   {precision_maligno:.2%} (Dos que chamamos de câncer, X% eram mesmo)")
print("-" * 40)

# Análise de Negócios
print("\n📝 ANÁLISE:")
if recall_maligno < 0.90:
    print(f"⚠️ ALERTA: Estamos deixando passar {(1-recall_maligno):.1%} dos casos de câncer!")
    print("   Sugestão: Diminuir o threshold (limiar) ou ajustar pesos.")
else:
    print("✅ O modelo tem alta sensibilidade para detectar a doença.")

In [0]:
# Drop 'features_raw' from train_data and test_data if it exists
if 'features_raw' in train_data.columns:
    train_data = train_data.drop('features_raw')
if 'features_raw' in test_data.columns:
    test_data = test_data.drop('features_raw')

from pyspark.ml.feature import PolynomialExpansion

# 1. CRIANDO O NOVO COMPONENTE: EXPANSÃO POLINOMIAL
poly_expansion = PolynomialExpansion(
    inputCol="features_raw", 
    outputCol="poly_features", 
    degree=2
)

# 2. ATUALIZANDO O SCALER
scaler_poly = StandardScaler(
    inputCol="poly_features",  # Agora a entrada é a versão polinomial
    outputCol="features_scaled",
    withStd=True, 
    withMean=True
)

# 3. O NOVO PIPELINE (NÃO-LINEAR)
pipeline_nlinear = Pipeline(stages=[
    assembler,       # Junta as 30 colunas
    poly_expansion,  # Cria interações (X^2, X*Y) - AQUI ESTÁ A "NÃO LINEARIDADE"
    scaler_poly,     # Padroniza tudo
    svm              # O mesmo LinearSVC (que agora vai aprender curvas)
])

# Precisamos atualizar a coluna de features do SVM para ler "features_scaled"
svm.setFeaturesCol("features_scaled")

# 4. TREINAMENTO E AVALIAÇÃO COMPARATIVA
print("⏳ Treinando SVM com Expansão Polinomial (Grau 2)...")
model_nlinear = pipeline_nlinear.fit(train_data)
predictions_nlinear = model_nlinear.transform(test_data)

# Avaliação
acc_nlinear = evaluator_acc.evaluate(predictions_nlinear)
f1_nlinear = evaluator_f1.evaluate(predictions_nlinear)

# Cálculo do recall para a classe 'Maligno' (1)
TP_nlinear = predictions_nlinear.filter((F.col("label") == 1) & (F.col("prediction") == 1)).count()
FN_nlinear = predictions_nlinear.filter((F.col("label") == 1) & (F.col("prediction") == 0)).count()
recall_nlinear = TP_nlinear / (TP_nlinear + FN_nlinear) if (TP_nlinear + FN_nlinear) > 0 else 0

print("\n" + "="*40)
print(f"📊 RESULTADO FINAL (COMPARATIVO)")
print("="*40)
print(f"1. SVM Simples (Linear):     {accuracy:.2%}")
print(f"2. SVM Polinomial (Grau 2):  {acc_nlinear:.2%}")
print(f"⚖️  F1-Score (Equilíbrio): {f1_nlinear:.2%}")
print(f"🚨 Recall (Sensibilidade - Maligno): {recall_nlinear:.2%}")
print("-" * 40)
print("\n--- Matriz de Confusão ---")
# Agrupamos pelo Gabarito (label) e pelo Chute (prediction)
confusion_matrix_nlinear = predictions_nlinear.groupBy("label", "prediction").count() \
                              .orderBy("label", "prediction")

display(confusion_matrix_nlinear)
# Verificando se melhorou
if acc_nlinear > accuracy:
    print("✅ CONCLUSÃO: Os dados tinham relações Não-Lineares ocultas!")
    print("   A expansão polinomial capturou padrões que a reta ignorou.")
elif acc_nlinear < accuracy:
    print("⚠️ CONCLUSÃO: Ocorreu Overfitting (Sobreajuste).")
    print("   O modelo ficou complexo demais para poucos dados (Ruído).")
else:
    print("ℹ️ CONCLUSÃO: O problema é essencialmente Linear.")
    print("   A complexidade extra não trouxe ganho real.")

In [0]:
# --- FASE 7: O DESAFIO FINAL (SCIKIT-LEARN RBF) ---
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score

print("🔄 Convertendo dados do Spark para Pandas (Para usar Scikit-Learn)...")
df_pandas = df_clean.toPandas()

# 1. Preparação (Scikit-Learn Way)
X = df_pandas.drop("label", axis=1)
y = df_pandas["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler_sklearn = StandardScaler()
X_train_scaled = scaler_sklearn.fit_transform(X_train)
X_test_scaled = scaler_sklearn.transform(X_test)

# 2. Treinando o RBF (A "Mágica" do Livro)
print("🧠 Treinando SVM com Kernel RBF...")
svm_rbf = SVC(kernel="rbf", C=1.0, gamma="scale", random_state=42)
svm_rbf.fit(X_train_scaled, y_train)

# 3. Avaliação
y_pred_rbf = svm_rbf.predict(X_test_scaled)
acc_rbf = accuracy_score(y_test, y_pred_rbf)
f1_rbf = f1_score(y_test, y_pred_rbf)
recall_rbf = recall_score(y_test, y_pred_rbf)

print("\n" + "="*40)
print(f"🏆 PLACAR FINAL DOS MODELOS")
print("="*40)
print(f"1. Spark Linear:        95.10%")
print(f"2. Spark Polinomial:    97.90%")
print(f"3. Scikit Kernel RBF:   {acc_rbf:.2%}")
print(f"⚖️  F1-Score:           {f1_rbf:.2%}")
print(f"🚨 Recall (Maligno):    {recall_rbf:.2%}")
print("-" * 40)

# Ajuste da conclusão final: foco em recall

# Matriz de Confusão simplificada
print("\n--- Matriz de Confusão (Scikit-Learn RBF) ---")
cm_rbf = confusion_matrix(y_test, y_pred_rbf)
print(f"Benigno (0) previsto como Benigno (0): {cm_rbf[0,0]}")
print(f"Maligno (1) previsto como Maligno (1): {cm_rbf[1,1]}")
print(f"Erros (Benigno→Maligno): {cm_rbf[0,1]}, (Maligno→Benigno): {cm_rbf[1,0]}")